In [ ]:
#Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import json
import git
from datetime import datetime
from flask import Flask, render_template, jsonify, request
from werkzeug.utils import secure_filename
from sklearn.metrics import classification_report

app = Flask(__name__, static_folder='static', template_folder='templates')


In [ ]:
#Configure variables for adversarial results
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'json', 'csv'}
REPO_PATH = 'adversarial_results' 
GITHUB_URL = 'https://github.com/UniSA-ICT-2025-SP1-P2/capstone_project/tree/master' #replace with correct URL once files created

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(REPO_PATH, exist_ok=True)
os.makedirs(os.path.join('static', 'viz'), exist_ok=True)

In [3]:
#Helper functions
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def load_results(file_path):
    """Load results from JSON or CSV file"""
    if file_path.endswith('.json'):
        with open(file_path, 'r') as f:
            return json.load(f)
    elif file_path.endswith('.csv'):
        return pd.read_csv(file_path).to_dict('records')
    return None

def sync_with_github():
    """Pull latest results from GitHub and push new results"""
    try:
        # Initialize repo if not exists
        if not os.path.exists(os.path.join(REPO_PATH, '.git')):
            repo = git.Repo.init(REPO_PATH)
            origin = repo.create_remote('origin', GITHUB_URL)
        else:
            repo = git.Repo(REPO_PATH)
            origin = repo.remote('origin')
        
        # Pull latest changes
        origin.pull()
        
        # Add all files, commit and push
        repo.git.add(all=True)
        commit_message = f"Update results - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        repo.git.commit('-m', commit_message)
        origin.push()
        return True
    except Exception as e:
        print(f"GitHub sync error: {str(e)}")
        return False

In [ ]:
#Function to generate metrics table
def generate_metrics_table(results_data):
    """Generate HTML table for metrics"""
    # Create DataFrame for table
    table_data = []
    
    for result in results_data:
        model_name = result.get('model_name', 'Unknown')
        
        # Baseline metrics
        if 'baseline_metrics' in result:
            metrics = result['baseline_metrics']
            table_data.append({
                'Model': model_name,
                'Training': 'Baseline',
                'Precision': metrics.get('precision', 0),
                'Recall': metrics.get('recall', 0),
                'F1-Score': metrics.get('f1_score', 0),
                'Support': metrics.get('support', 0),
                'Accuracy': result.get('standard_accuracy', 0)
            })
        
        # PGD metrics
        if 'pgd_metrics' in result:
            metrics = result['pgd_metrics']
            table_data.append({
                'Model': model_name,
                'Training': 'PGD',
                'Precision': metrics.get('precision', 0),
                'Recall': metrics.get('recall', 0),
                'F1-Score': metrics.get('f1_score', 0),
                'Support': metrics.get('support', 0),
                'Accuracy': result.get('pgd_accuracy', 0)
            })
        
        # FGSM metrics
        if 'fgsm_metrics' in result:
            metrics = result['fgsm_metrics']
            table_data.append({
                'Model': model_name,
                'Training': 'FGSM',
                'Precision': metrics.get('precision', 0),
                'Recall': metrics.get('recall', 0),
                'F1-Score': metrics.get('f1_score', 0),
                'Support': metrics.get('support', 0),
                'Accuracy': result.get('fgsm_accuracy', 0)
            })
    
    df = pd.DataFrame(table_data)
    
    # Save as CSV for download
    metrics_csv = os.path.join('static', 'metrics_table.csv')
    df.to_csv(metrics_csv, index=False)
    
    # Generate HTML table
    html_table = df.to_html(classes='table table-striped table-bordered', index=False)
    
    return html_table, 'metrics_table.csv'

In [ ]:
#Function to create visualisations
def generate_visualisations(results_data):
    """Generate visualisation images from results data"""
    output_files = []
    
    # Clean data directory
    viz_dir = os.path.join('static', 'viz')
    os.makedirs(viz_dir, exist_ok=True)
    
    # Extract model names
    model_names = []
    for result in results_data:
        model_names.append(result.get('model_name', 'Unknown'))
    model_names = list(dict.fromkeys(model_names))  # Remove duplicates
    
    # 1. Accuracy comparison chart
    plt.figure(figsize=(12, 7))
    
    # Extract data for different scenarios
    standard_acc = []
    pgd_acc = []
    fgsm_acc = []
    
    for model in model_names:
        model_results = [r for r in results_data if r.get('model_name') == model]
        if model_results:
            result = model_results[0]  # Take the first result for this model
            standard_acc.append(result.get('standard_accuracy', 0))
            pgd_acc.append(result.get('pgd_accuracy', 0))
            fgsm_acc.append(result.get('fgsm_accuracy', 0))
    
    # Plot grouped bar chart
    x = np.arange(len(model_names))
    width = 0.25
    
    plt.bar(x - width, standard_acc, width, label='Baseline')
    plt.bar(x, pgd_acc, width, label='After PGD Training')
    plt.bar(x + width, fgsm_acc, width, label='After FGSM Training')
    
    plt.xlabel('Models')
    plt.ylabel('Accuracy (%)')
    plt.title('Model Accuracy Comparison')
    plt.xticks(x, model_names, rotation=45)
    plt.ylim(0, 100)
    plt.legend()
    plt.tight_layout()
    
    # Save the plot
    accuracy_plot = os.path.join(viz_dir, 'accuracy_comparison.png')
    plt.savefig(accuracy_plot)
    output_files.append('viz/accuracy_comparison.png')
    plt.close()
    
    # 2. Precision comparison
    plt.figure(figsize=(12, 7))
    
    baseline_prec = []
    pgd_prec = []
    fgsm_prec = []
    
    for model in model_names:
        model_results = [r for r in results_data if r.get('model_name') == model]
        if model_results:
            result = model_results[0]
            
            baseline_metrics = result.get('baseline_metrics', {})
            pgd_metrics = result.get('pgd_metrics', {})
            fgsm_metrics = result.get('fgsm_metrics', {})
            
            baseline_prec.append(baseline_metrics.get('precision', 0))
            pgd_prec.append(pgd_metrics.get('precision', 0))
            fgsm_prec.append(fgsm_metrics.get('precision', 0))
    
    plt.bar(x - width, baseline_prec, width, label='Baseline')
    plt.bar(x, pgd_prec, width, label='After PGD Training')
    plt.bar(x + width, fgsm_prec, width, label='After FGSM Training')
    
    plt.xlabel('Models')
    plt.ylabel('Precision')
    plt.title('Model Precision Comparison')
    plt.xticks(x, model_names, rotation=45)
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    
    precision_plot = os.path.join(viz_dir, 'precision_comparison.png')
    plt.savefig(precision_plot)
    output_files.append('viz/precision_comparison.png')
    plt.close()
    
    # 3. Recall comparison
    plt.figure(figsize=(12, 7))
    
    baseline_recall = []
    pgd_recall = []
    fgsm_recall = []
    
    for model in model_names:
        model_results = [r for r in results_data if r.get('model_name') == model]
        if model_results:
            result = model_results[0]
            
            baseline_metrics = result.get('baseline_metrics', {})
            pgd_metrics = result.get('pgd_metrics', {})
            fgsm_metrics = result.get('fgsm_metrics', {})
            
            baseline_recall.append(baseline_metrics.get('recall', 0))
            pgd_recall.append(pgd_metrics.get('recall', 0))
            fgsm_recall.append(fgsm_metrics.get('recall', 0))
    
    plt.bar(x - width, baseline_recall, width, label='Baseline')
    plt.bar(x, pgd_recall, width, label='After PGD Training')
    plt.bar(x + width, fgsm_recall, width, label='After FGSM Training')
    
    plt.xlabel('Models')
    plt.ylabel('Recall')
    plt.title('Model Recall Comparison')
    plt.xticks(x, model_names, rotation=45)
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    
    recall_plot = os.path.join(viz_dir, 'recall_comparison.png')
    plt.savefig(recall_plot)
    output_files.append('viz/recall_comparison.png')
    plt.close()
    
    # 4. F1-Score comparison
    plt.figure(figsize=(12, 7))
    
    baseline_f1 = []
    pgd_f1 = []
    fgsm_f1 = []
    
    for model in model_names:
        model_results = [r for r in results_data if r.get('model_name') == model]
        if model_results:
            result = model_results[0]
            
            baseline_metrics = result.get('baseline_metrics', {})
            pgd_metrics = result.get('pgd_metrics', {})
            fgsm_metrics = result.get('fgsm_metrics', {})
            
            baseline_f1.append(baseline_metrics.get('f1_score', 0))
            pgd_f1.append(pgd_metrics.get('f1_score', 0))
            fgsm_f1.append(fgsm_metrics.get('f1_score', 0))
    
    plt.bar(x - width, baseline_f1, width, label='Baseline')
    plt.bar(x, pgd_f1, width, label='After PGD Training')
    plt.bar(x + width, fgsm_f1, width, label='After FGSM Training')
    
    plt.xlabel('Models')
    plt.ylabel('F1-Score')
    plt.title('Model F1-Score Comparison')
    plt.xticks(x, model_names, rotation=45)
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    
    f1_plot = os.path.join(viz_dir, 'f1_comparison.png')
    plt.savefig(f1_plot)
    output_files.append('viz/f1_comparison.png')
    plt.close()
    
    # 5. Radar chart for all metrics (for each model)
    for model in model_names:
        model_results = [r for r in results_data if r.get('model_name') == model]
        if model_results:
            result = model_results[0]
            
            baseline_metrics = result.get('baseline_metrics', {})
            pgd_metrics = result.get('pgd_metrics', {})
            fgsm_metrics = result.get('fgsm_metrics', {})
            
            # Extract metrics
            labels = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
            
            baseline_values = [
                result.get('standard_accuracy', 0)/100,  # Normalize accuracy to 0-1
                baseline_metrics.get('precision', 0),
                baseline_metrics.get('recall', 0),
                baseline_metrics.get('f1_score', 0)
            ]
            
            pgd_values = [
                result.get('pgd_accuracy', 0)/100,  # Normalize accuracy to 0-1
                pgd_metrics.get('precision', 0),
                pgd_metrics.get('recall', 0),
                pgd_metrics.get('f1_score', 0)
            ]
            
            fgsm_values = [
                result.get('fgsm_accuracy', 0)/100,  # Normalize accuracy to 0-1
                fgsm_metrics.get('precision', 0),
                fgsm_metrics.get('recall', 0),
                fgsm_metrics.get('f1_score', 0)
            ]
            
            # Number of variables
            N = len(labels)
            
            # Create angles for each variable
            angles = [n / float(N) * 2 * np.pi for n in range(N)]
            angles += angles[:1]  # Close the loop
            
            # Extend the values to close the loop
            baseline_values += baseline_values[:1]
            pgd_values += pgd_values[:1]
            fgsm_values += fgsm_values[:1]
            
            # Create the plot
            fig = plt.figure(figsize=(10, 8))
            ax = fig.add_subplot(111, polar=True)
            
            # Draw the outline of our data
            ax.plot(angles, baseline_values, 'o-', linewidth=2, label='Baseline')
            ax.fill(angles, baseline_values, alpha=0.1)
            
            ax.plot(angles, pgd_values, 'o-', linewidth=2, label='PGD Training')
            ax.fill(angles, pgd_values, alpha=0.1)
            
            ax.plot(angles, fgsm_values, 'o-', linewidth=2, label='FGSM Training')
            ax.fill(angles, fgsm_values, alpha=0.1)
            
            # Fix axis to go in the right order and start at 12 o'clock
            ax.set_theta_offset(np.pi / 2)
            ax.set_theta_direction(-1)
            
            # Draw axis lines for each angle and label
            ax.set_xticks(angles[:-1])
            ax.set_xticklabels(labels)
            
            # Set y-axis limit
            ax.set_ylim(0, 1)
            
            # Add legend
            plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
            
            plt.title(f"{model} - All Metrics Comparison")
            
            # Save the plot
            radar_plot = os.path.join(viz_dir, f'radar_{model.replace(" ", "_")}.png')
            plt.savefig(radar_plot)
            output_files.append(f'viz/radar_{model.replace(" ", "_")}.png')
            plt.close()
    
    return output_files

In [ ]:
#Routes
@app.route('/')
def index():
    """Main page with visualisation dashboard"""
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    """Handle file uploads with results data"""
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        
        # Also save to Git repo
        repo_file_path = os.path.join(REPO_PATH, 'data', filename)
        os.makedirs(os.path.dirname(repo_file_path), exist_ok=True)
        with open(file_path, 'rb') as src_file:
            with open(repo_file_path, 'wb') as dst_file:
                dst_file.write(src_file.read())
        
        # Load results and generate visualizations
        results_data = load_results(file_path)
        viz_files = generate_visualisations(results_data)
        
        # Generate metrics table
        metrics_table, metrics_csv = generate_metrics_table(results_data)
        
        # Save metrics table to file
        metrics_html = os.path.join('static', 'metrics_table.html')
        with open(metrics_html, 'w') as f:
            f.write(metrics_table)
        
        # Sync with GitHub
        sync_success = sync_with_github()
        
        return jsonify({
            'success': True,
            'message': 'File uploaded and processed successfully',
            'github_sync': sync_success,
            'visualisations': viz_files,
            'metrics_table': metrics_table,
            'metrics_csv': metrics_csv
        })
    
    return jsonify({'error': 'Invalid file type'}), 400

@app.route('/results')
def get_results():
    """Return all results data as JSON"""
    results = []
    
    # Get data from repo
    data_dir = os.path.join(REPO_PATH, 'data')
    if os.path.exists(data_dir):
        for filename in os.listdir(data_dir):
            if allowed_file(filename):
                file_path = os.path.join(data_dir, filename)
                data = load_results(file_path)
                if data:
                    if isinstance(data, list):
                        results.extend(data)
                    else:
                        results.append(data)
    
    return jsonify(results)

@app.route('/sync', methods=['POST'])
def github_sync():
    """Force sync with GitHub repository"""
    success = sync_with_github()
    return jsonify({'success': success})

@app.route('/demo')
def demo():
    """Generate demo data for visualisation"""
    demo_data = [
        {
            'model_name': 'RandomForest',
            'standard_accuracy': 94.5,
            'pgd_accuracy': 85.2,
            'fgsm_accuracy': 88.8,
            'baseline_metrics': {
                'precision': 0.95,
                'recall': 0.94,
                'f1_score': 0.945,
                'support': 1000
            },
            'pgd_metrics': {
                'precision': 0.86,
                'recall': 0.85,
                'f1_score': 0.855,
                'support': 1000
            },
            'fgsm_metrics': {
                'precision': 0.89,
                'recall': 0.88,
                'f1_score': 0.885,
                'support': 1000
            }
        },
        {
            'model_name': 'LogisticRegression',
            'standard_accuracy': 93.7,
            'pgd_accuracy': 82.1,
            'fgsm_accuracy': 86.9,
            'baseline_metrics': {
                'precision': 0.94,
                'recall': 0.93,
                'f1_score': 0.935,
                'support': 1000
            },
            'pgd_metrics': {
                'precision': 0.83,
                'recall': 0.82,
                'f1_score': 0.825,
                'support': 1000
            },
            'fgsm_metrics': {
                'precision': 0.87,
                'recall': 0.86,
                'f1_score': 0.865,
                'support': 1000
            }
        },
        {
            'model_name': 'Neural Network',
            'standard_accuracy': 96.2,
            'pgd_accuracy': 88.5,
            'fgsm_accuracy': 91.3,
            'baseline_metrics': {
                'precision': 0.96,
                'recall': 0.96,
                'f1_score': 0.96,
                'support': 1000
            },
            'pgd_metrics': {
                'precision': 0.89,
                'recall': 0.88,
                'f1_score': 0.885,
                'support': 1000
            },
            'fgsm_metrics': {
                'precision': 0.92,
                'recall': 0.91,
                'f1_score': 0.915,
                'support': 1000
            }
        }
    ]
    
    # Save demo data
    demo_file = os.path.join(app.config['UPLOAD_FOLDER'], 'demo_data.json')
    with open(demo_file, 'w') as f:
        json.dump(demo_data, f)
    
    # Generate visualizations
    viz_files = generate_visualisations(demo_data)
    
    # Generate metrics table
    metrics_table, metrics_csv = generate_metrics_table(demo_data)
    
    # Save metrics table to file
    metrics_html = os.path.join('static', 'metrics_table.html')
    with open(metrics_html, 'w') as f:
        f.write(metrics_table)
    
    return jsonify({
        'success': True,
        'message': 'Demo data generated',
        'visualisations': viz_files,
        'metrics_table': metrics_table,
        'metrics_csv': metrics_csv
    })

